# Jupyter Notebookのセットアップ

## 前提

このnotebookでは、Open OnDemandからJupyter Notebookのサーバをジョブとして起動し、それに対して接続する設定を行います。設定にあたっては、以下を前提とします。

* [030-ジョブ実行環境の設定](./030-ジョブ実行環境の設定.ipynb)のnotebookによりOpen OnDemandの設定を完了し、Open OnDemandからジョブ実行が可能になっていること。
* このnotebookで使用するJupyter Notebookサーバは、[NII-cloud-operation/Jupyter-LC_docker](https://github.com/NII-cloud-operation/Jupyter-LC_docker/)のDockerイメージをベースとし、Open OnDemandから使用可能になるよう変更したものとする。Open OnDemandから使用する際には、DockerイメージをSingularityコンテナとして実行する。

## 準備

### group_varsの読み込み

group_varsの読み込みに先立ち、ユニットグループ名をチェックします。

In [ ]:
!ls -1 group_vars/*.yml | sed -e 's/^group_vars\///' -e 's/\.yml//' | sort

`ugroup_name`にユニットグループ名を設定します。

In [ ]:
ugroup_name = 

group_varsを読み込みます。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)

### ansibleの動作確認

ansibleでSlurmクラスタにアクセスできることを確認します。

In [ ]:
!ansible -m ping {ugroup_name}

ansibleで指定するノード集合を設定します。

In [ ]:
target_master = f'{ugroup_name}_master'
target_compute = f'{ugroup_name}_compute'

## 必要パッケージのインストール

空きポートの検索やNotebookサーバの起動確認のために`nc`コマンドが使われるため、nmap-ncatパッケージをインストールします。

In [ ]:
!ansible {target_master} -m dnf -b -a \
    "name=nmap-ncat state=present"

## Notebookサーバイメージの配置

[NII-cloud-operation/Jupyter-LC_docker](https://github.com/NII-cloud-operation/Jupyter-LC_docker/)のDockerイメージをベースとしてSingularityイメージを作成します。

Singularityイメージは、`/opt/ohpc/pub/apps/jupyter/notebook.sif`として配置します。

In [ ]:
nb_image_dir = '/opt/ohpc/pub/apps/jupyter'
nb_image = 'notebook.sif'

イメージを配置するディレクトリを作成します。

In [ ]:
!ansible {target_master} -m file -b -a \
    "path={nb_image_dir} state=directory"

OpenHPCではSingularityがインストール済みであるため、Singuiarityイメージは、マスターノードで作成します。そのため、まずSingularityイメージ作成に必要なファイルをマスターノードに転送します。

In [ ]:
!ansible {target_master} -m copy -b -a \
    "src=./singularity/jupyter dest={nb_image_dir}"

Singularityイメージをビルドします。ビルドには数分程度かかります。

In [ ]:
!ansible {target_master} -m shell -b -a \
    'bash -c "source ~/.bashrc && \
    rm -f {nb_image_dir}/{nb_image} && \
    module load singularity && \
    cd {nb_image_dir}/jupyter && \
    singularity build ../notebook.sif notebook.def"'

イメージをビルドするために転送したファイルを削除します。

In [ ]:
!ansible {target_master} -m file -b -a \
    "path={nb_image_dir}/jupyter state=absent"

## Web UI関連ファイルの配置

Open OnDemandのWeb UIを経由したJupyter Notebookジョブの投入に必要となるファイルを配置します。この節で配置しているファイルは、 https://github.com/OSC/bc_example_jupyter のものをベースとして変更を加えているか、変更せずそのまま用いています。変更不要のものについては、直接マスターノードからダウンロードして配置しています。ベースとしている各ファイルのライセンスについては、[LICENSE.txt](https://github.com/OSC/bc_example_jupyter/blob/master/LICENSE.txt)にある通りです。

### Appディレクトリの作成

Open OnDemandにWeb UIで統合されるアプリケーションは、マスターノードの`/var/www/ood/apps/sys`以下のディレクトリに配置します。

Jupyter Notebook向けのディレクトリを作成します。

In [ ]:
nb_app_dir = '/var/www/ood/apps/sys/jupyter'

In [ ]:
!ansible {target_master} -m file -b -a \
    "path={nb_app_dir} state=directory mode=0755"

### `form.yml`

`form.yml`は、主に、Notebookサーバをジョブとして実行する際のパラメータなどをWebフォームから設定可能とするためのファイルです。ここでは、`form.yml`を生成するのに必要な設定を行い、生成したファイルをマスターノードに配置します。

ジョブを投入するクラスタ名を、`ood_nb_clst`に設定します。[030-ジョブ実行環境の設定](./030-ジョブ実行環境の設定.ipynb)のnotebook通りに設定した場合には、以下の設定をそのまま使用できます。変更した場合は、それに合わせて以下のセルを変更してください。

In [ ]:
ood_nb_clst = f'{ugroup_name}_cluster'

クラスタ定義ファイルの存在を確認することで、上の設定が正しいことを確認します。セルの実行結果がエラーとなる場合は、上の設定が誤っている(対応するクラスタ定義ファイルが存在しない)ので修正が必要です。

In [ ]:
!ansible {target_master} -m file -b -a \
    "path=/etc/ood/config/clusters.d/{ood_nb_clst}.yml state=file"

`form.yml`のベースとなるデータ構造を作成します。`attributes.modules`に`singularity`を指定することで、ジョブスクリプトでSingularityを実行可能とします。また、`form`には、Web UIからユーザが設定可能なパラメータを指定します。詳細については、[Open OnDemandの対応するページ](https://osc.github.io/ood-documentation/latest/app-development/tutorials-interactive-apps/add-jupyter/customize-attributes.html)や[GitHubのform.yml](https://github.com/OSC/bc_example_jupyter/blob/master/form.yml)のコメントを参照してください。

In [ ]:
form_yml_data = {
    'cluster': ood_nb_clst,
    'attributes': {
        'modules': 'singularity'
    },
    'form': [
        'modules',
        'bc_queue',
        'bc_num_hours',
        'bc_num_slots'
    ]
}

`form_yml_data`をYAML形式に変換します。

In [ ]:
import yaml

form_yml = yaml.safe_dump(form_yml_data, default_flow_style=False)

`form.yml`を配置します。

In [ ]:
!ansible {target_master} -m copy -b -a \
    "content='{form_yml}' dest={nb_app_dir}/form.yml mode=0644"

### `submit.yml.erb`

`submit.yml.erb`は、ジョブ投入の際のパラメータを設定するためのファイルです。

ジョブに割り当てるスロット数を、Webフォームで設定した後となるよう、`script.native`に設定します(`sbatch`に与えるパラメータ)。

In [ ]:
submit_yml_data = {
    'script': {
        'native': [
            '-N',
            '<%= bc_num_slots.blank? ? 1 : bc_num_slots.to_i %>'
        ]
    }
}

OpenHPC環境においてOpen OnDemandからのジョブ投入が正しく動作するよう、ジョブ投入時の環境変数を引き継ぐ指定をします(`script.copy_environment`に`True`を指定)。

In [ ]:
submit_yml_data['script'].update(
    {
        'copy_environment': True
    }
)

batch connect(WebアプリケーションをOpen OnDemandからジョブとして実行するためのフレームワーク)のテンプレートを設定します。

In [ ]:
submit_yml_data['batch_connect'] = {
    'template': 'basic'
}

`submit.yml.erb`を配置します。

In [ ]:
with open('./submit.yml.erb', 'w') as f:
    yaml.safe_dump(submit_yml_data, f, default_flow_style=False)

!ansible {target_master} -m copy -b -a \
    "src=./submit.yml.erb dest={nb_app_dir}/submit.yml.erb mode=0644"
!rm -f ./submit.yml.erb

### ジョブスクリプトテンプレート

Jupyter NotebookをSingularityコンテナとして実行するよう変更したジョブスクリプトテンプレートを配置します。

In [ ]:
!ansible {target_master} -m file -b -a \
    "path={nb_app_dir}/template state=directory mode=0755"

tmpl_files = [
    'before.sh.erb',
    'script.sh.erb'
]
for f in tmpl_files:
    !ansible {target_master} -m copy -b -a \
        "src=ood-template/{f} dest={nb_app_dir}/template/{f} mode=0755"

`script.sh.erb`でNotebookサーバイメージを実行しているため、[Notebookサーバイメージの配置](#Notebookサーバイメージの配置)でイメージの配置場所を変更した場合には、`script.sh.erb`の内容の変更が必要です。

### その他

カスタマイズせず使用できるファイルを、 https://github.com/OSC/bc_example_jupyter/ からダウンロードして配置します。

In [ ]:
unmod_files = [
    'icon.png',
    'manifest.yml',
    'view.html.erb',
    'template/after.sh'
]

baseurl = 'https://github.com/OSC/bc_example_jupyter/raw/master'
for f in unmod_files:
    !ansible {target_master} -m shell -b -a \
        "curl -L {baseurl}/{f} > {nb_app_dir}/{f}"

!ansible {target_master} -b -a \
    "chmod +x {nb_app_dir}/template/after.sh"

## リバースプロキシの設定

Open OnDemandでは、計算ノードでジョブとして実行されるWebアプリケーションを、Apache httpdのリバースプロキシ機能を用いて外部接続可能とします。このため、`/etc/ood/config/ood_portal.yml`を更新します。

`ood_portal.yml`を編集するため、現状のファイルをマスターノードから取得します。

In [ ]:
!ansible {target_master} -m fetch -b -a \
    "src=/etc/ood/config/ood_portal.yml \
    dest=./ flat=true"

`ood_portal.yml`を読み込みます。

In [ ]:
import yaml

with open('ood_portal.yml') as f:
    ood_portal = yaml.safe_load(f)

`ood_portal.yml`の`node_uri`に、アクセスされたURIをリバースプロキシにより転送するためのプレフィックスを`/node`と指定します。これにより、`/node/<host>/<port>/...`に対するリクエストが、`<host>:<port>`に転送されます(転送時には、URIの`/node/<host>/<port>`部分はそのまま転送されます)。

In [ ]:
ood_portal['node_uri'] = '/node'

`host_regex`に、リクエスト転送先のホストを限定するための正規表現を設定します。

`host_regex`設定のため、計算ノードのホスト名のプレフィックスを推定します。

In [ ]:
cn_present = gvars['compute_etc_hosts']
cn_max = max(cn_present.values(), key=len)
for i in range(len(cn_max) - 2):
    c_hostname_prefix = cn_max[0:-(i+1)]
    for n in list(cn_present.values()):
        matched = n.startswith(c_hostname_prefix)
        if not matched:
            break
    if matched:
        break

c_hostname_prefix

表示されたプレフィックスが正しくない場合は、以下のセルのコメントを外して、c_hostname_prefixに正しいプレフィックスを設定してください。

In [ ]:
# c_hostname_prefix = ''

計算ノードのホスト名のプレフィックスに基づき、`host_regex`を設定します。

In [ ]:
ood_portal['host_regex'] = f'{c_hostname_prefix}\d+'

ローカルの`ood_portal.yml`を更新します。

In [ ]:
with open('ood_portal.yml', 'w') as f:
    yaml.safe_dump(ood_portal, f, default_flow_style=False)

更新後の内容を確認します。

In [ ]:
!cat ood_portal.yml

内容が正しいことを確認したら、マスターノードに転送します。

In [ ]:
!ansible {target_master} -m copy -b -a \
    "src=ood_portal.yml dest=/etc/ood/config/ood_portal.yml"

`ood_portal.yml`の更新を、httpdの設定ファイルなどに反映させるため、`update-ood-portal`を実行します。

In [ ]:
!ansible {target_master} -b -a \
    "/opt/ood/ood-portal-generator/sbin/update_ood_portal"

`httpd.service`, `htcacheclean.service`をrestartします。

In [ ]:
!ansible {target_master} -m systemd -b -a \
    "name=httpd.service state=restarted"
!ansible {target_master} -m systemd -b -a \
    "name=htcacheclean.service state=restarted"

以上でOpen OnDemandからJupyter Notebookサーバをジョブとして実行し、Open OnDemandのWeb UIからサーバに接続できるようになります。